# TP outil d’estimation de véhicules d’occasions

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

## Libraries import

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from datetime import datetime
import seaborn as sns
#from sklearn.model_selection import train_test_split

## Dataframe import

In [3]:
fpath = os.path.join('Data_cars.csv')
print(fpath)
df = pd.read_csv(fpath)

Data_cars.csv


FileNotFoundError: [Errno 2] File b'Data_cars.csv' does not exist: b'Data_cars.csv'

## Dataframe exploration

In [ ]:
df.shape

In [ ]:
df.columns.tolist()

In [ ]:
df.head(10)

In [ ]:
print (df.dtypes)

In [ ]:
df['Mileage'] = df['Mileage'].map(lambda x : int(float(x[:-3])))
print (df.dtypes)

In [ ]:
df.at[2,'Description']

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
plt.figure(figsize=(50,20))
plt.scatter(df['Mileage'],df['Price'])
plt.xlabel("Price",size=40)
plt.xticks(size=35)
plt.ylabel("Mileage",size=40)
plt.yticks(size=35)
plt.title('Pice in terms of Mileage',size=35)

In [ ]:
plt.hist(df['Price'],bins=500)
plt.xlabel("price",size=10)
plt.ylabel("Number of cars",size=10)

In [ ]:
df2 = df.loc[df['Price'] < 100000]
plt.hist(df2['Price'],bins=500)
plt.xlabel("price",size=10)
plt.ylabel("Number of cars",size=10)

In [ ]:
df['Make']=df['Make'].map(lambda x: x.strip())
df.Make.unique()

In [ ]:
#pd.set_option('display.max_rows', None)
df2=df.Make.value_counts()#normalize=True
df2=pd.Series.to_frame(df2)
df2.reset_index(inplace=True)
df2.columns=["Maker", "nbrCar"]
df2

In [ ]:
plt.figure(figsize=(50,20))
plt.bar("Maker", "nbrCar",data = df2)
plt.xlabel("Mark",size=30)
plt.xticks(rotation=90,size=20)
plt.ylabel("Number of cars",size=30)
plt.yticks(size=20)
plt.title('Number of cars per Mark',size=35)

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.gca()
df.boxplot(column='Price', ax=ax)
# frame['ArrDelay'].plot.box(ax=ax) # Alternative
ax.set_title('BoxPlot of Price')
ax.set_ylabel('€')

In [ ]:
pd.options.display.max_rows = 15
df.sort_values(by=['Model_year'])
df.loc[df['Model_year'] > 2020]

In [ ]:
df=df.loc[df['Model_year'] < 2020]

In [ ]:
df=df.loc[df['Model_year'] > 1945]

In [ ]:
df.at[56,'Description']

In [ ]:
df['Fiscal_Horsepower'] = df['Description'].map(lambda x : int(re.search(r'puissance_fiscale: (.*?), ',x).group(1)))
df['Fiscal_Horsepower']

In [ ]:
df['Fiscal_Horsepower'].unique()

In [ ]:
df['doors'] = df['Description'].map(lambda x : re.search(r'portes:(.*?), ',x).group(1))
df['doors'] = df['doors'].map(lambda x: x.strip())
df['doors'] = df['doors'].map(lambda x: 7  if(x=='') else int(x[:-2]))
df['doors'].unique()

In [ ]:
df['Options'] = df['Description'].map(lambda x : re.search(r'options:(.*?), couleur:', x).group(1).split(';'))
df['Options_Count'] = df['Description'].map(lambda x : re.search(r'options:(.*?), couleur:', x).group(1).count(';'))
df['Couleurs'] = df['Description'].map(lambda x : re.split(r'couleur: ', x)[-1])
df['Couleurs'].unique()

In [ ]:
df['Age']  = df['Online'].map(lambda x : datetime.strptime(x, '%d/%m/%Y à %Hh%M'))
df['Age'] = df.apply(lambda row: row['Age'].year - int(row['Model_year']), axis=1)

In [ ]:
df.head()

In [ ]:
df['Fuel'].unique()

In [ ]:
df = df.join(pd.get_dummies(df['Fuel']))
df.head()

In [ ]:
pd.set_option('display.max_columns', None)
df = df.join(pd.get_dummies(df['Gearbox']))
df.head()

In [ ]:
pd.set_option('display.max_rows', None)
print (df.dtypes)
pd.set_option('display.max_rows', 15)

In [ ]:
def cylindre(string):
    cyl = re.search(r'(\d*\.\d*)', string)
    if(cyl != None):
        if cyl.group(1) != '.':
            val = round(float(cyl.group(1)), 1)
            return val
        else:
            return 0
    else:
        return 0
    
df['Cylindre'] = df['Description'].map(lambda x :  cylindre(re.search(r'version: (.*?),', x).group(1)))

In [ ]:
sns.set()
plt.subplots(figsize=(20,15))
sns.heatmap(df.corr().round(2), annot=True)

In [ ]:
df = df.join(pd.get_dummies(df['Cylindre']))

In [ ]:
df.head()

In [ ]:
df['Cylindre'].unique()

In [ ]:
df['Cylindre'].value_counts().nlargest(40).plot(kind='bar', figsize=(10,5))
plt.title('Cylindrées')
plt.ylabel('Fréquence')
plt.xlabel('Cylindrée')

In [ ]:
df = df.join(pd.get_dummies(df['Make']))
list(df)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
pd.set_option('display.max_rows', None)
print (df.dtypes)
pd.set_option('display.max_rows', 15)

In [ ]:
pd.set_option('display.max_columns', None)
df.columns

In [ ]:
input_columns=['Model_year','Age','Mileage','Fiscal_Horsepower','Cylindre', 'Options_Count','doors', 'Bicarburation essence GNV','Bicarburation essence GPL','Bicarburation essence bioéthanol','Diesel', 'Electrique','Hybride diesel électrique','Hybride essence électrique','automatique','mécanique']

In [ ]:
X = df[input_columns]
#independent variables
y = df['Price'] #dependent (target) variable

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = 0.2 )

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
linreg = LinearRegression()
linreg.fit(X_train, y_train) 

In [ ]:
linreg.score(X_train, y_train)


In [ ]:
linreg.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
regr = RandomForestRegressor(max_depth=5, random_state=0, n_estimators=10)

print('R-squared score (training): {:.3f}'
     .format(regr.score(X_train, y_train)))

print('R-squared score (training): {:.3f}'
     .format(regr.score(X_test, y_test)))

In [ ]:
regr = RandomForestRegressor(max_depth=20, random_state=0, n_estimators=10)

regr.fit(X_train, y_train)
print('R-squared score (training): {:.3f}'
     .format(regr.score(X_train, y_train)))

print('R-squared score (training): {:.3f}'
     .format(regr.score(X_test, y_test)))

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
score = mean_absolute_percentage_error(0.954,0.874)
print("The percentage : {} %".format(score))

# NLP Processing

In [ ]:
df['Options'] = df['Options'].map(lambda x : re.sub('[,.\/!?]', '', x))
df['Options'] = df['Options'].map(lambda x : re.sub('\d', '', x))

In [ ]:
df['Options'] = df['Description'].map(lambda x : re.search(r’options:(.*?), couleur:’, x).group(1))
df['Options'] = df['Options'].map(lambda x : (x.replace(‘;’, ' ’))) 